In [1]:
import copy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

Data = pd.read_csv('/content/drive/MyDrive/FullData_train.csv')

In [4]:
Y = Data[['Object_Held']]

F = []
P = []

for elem in Data.columns:
  if elem[-1]=='e':
    F.append(elem)
  if elem[-1]=='n':
    P.append(elem)

X_position = copy.deepcopy(Data[P])
X_force = copy.deepcopy(Data[F])


In [5]:
X_force.shape

(166400, 16)

In [ ]:
import tensorflow as tf

input_shape = (16,)

encoder_inputs = tf.keras.Input(shape=input_shape)
HL1 = tf.keras.layers.Dense(32, activation='sigmoid')(encoder_inputs)
HL2 = tf.keras.layers.Dense(24, activation='sigmoid')(HL1)
HL3 = tf.keras.layers.Dense(24, activation='relu')(HL2)
HL4 = tf.keras.layers.Dense(18, activation='relu')(HL3)
HL5 = tf.keras.layers.Dense(10, activation='relu')(HL4)
encoder = tf.keras.Model(encoder_inputs, HL5)

decoder_inputs = tf.keras.Input(shape=(10,))
HL_D1 = tf.keras.layers.Dense(16, activation='sigmoid')(decoder_inputs)
HL_D2 = tf.keras.layers.Dense(16, activation='sigmoid')(HL_D1)
HL_D3 = tf.keras.layers.Dense(16, activation='sigmoid')(HL_D2)
decoder_outputs = tf.keras.layers.Dense(16, activation='linear')(HL_D3)
decoder = tf.keras.Model(decoder_inputs, decoder_outputs)

inputs = tf.keras.Input(shape=input_shape)
encoded = encoder(inputs)
decoded = decoder(encoded)
autoencoder = tf.keras.Model(inputs, decoded)

autoencoder.compile(optimizer='adam', loss='mse')


In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_force)
X_force_scaled = scaler.transform(X_force)

In [ ]:
autoencoder.fit(X_force_scaled, X_force_scaled, batch_size=3, epochs=1)
autoencoder.fit(X_force_scaled, X_force_scaled, batch_size=32, epochs=10)



55467/55467 [==============================] - 167s 3ms/step - loss: 0.2795
Epoch 1/10
5200/5200 [==============================] - 16s 3ms/step - loss: 0.1765
Epoch 2/10
5200/5200 [==============================] - 16s 3ms/step - loss: 0.1463
Epoch 3/10
5200/5200 [==============================] - 15s 3ms/step - loss: 0.1219
Epoch 4/10
5200/5200 [==============================] - 15s 3ms/step - loss: 0.1048
Epoch 5/10
5200/5200 [==============================] - 16s 3ms/step - loss: 0.0926
Epoch 6/10
5200/5200 [==============================] - 15s 3ms/step - loss: 0.0813
Epoch 7/10
5200/5200 [==============================] - 16s 3ms/step - loss: 0.0718
Epoch 8/10
5200/5200 [==============================] - 16s 3ms/step - loss: 0.0642
Epoch 9/10
5200/5200 [==============================] - 15s 3ms/step - loss: 0.0583
Epoch 10/10
5200/5200 [==============================] - 15s 3ms/step - loss: 0.0535


In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_position)
X_position_scaled = scaler.transform(X_position)

In [ ]:
import tensorflow as tf

input_shape = (16,)

encoder_inputs_pos = tf.keras.Input(shape=input_shape)
HL1_pos = tf.keras.layers.Dense(32, activation='sigmoid')(encoder_inputs_pos)
HL2_pos = tf.keras.layers.Dense(24, activation='sigmoid')(HL1_pos)
HL3_pos = tf.keras.layers.Dense(24, activation='relu')(HL2_pos)
HL4_pos = tf.keras.layers.Dense(18, activation='relu')(HL3_pos)
HL5_pos = tf.keras.layers.Dense(10, activation='relu')(HL4_pos)
encoder_pos = tf.keras.Model(encoder_inputs_pos, HL5_pos)

decoder_inputs_pos = tf.keras.Input(shape=(10,))
HL_D1_pos = tf.keras.layers.Dense(16, activation='sigmoid')(decoder_inputs_pos)
HL_D2_pos = tf.keras.layers.Dense(16, activation='sigmoid')(HL_D1_pos)
HL_D3_pos = tf.keras.layers.Dense(16, activation='sigmoid')(HL_D2_pos)
decoder_outputs_pos = tf.keras.layers.Dense(16, activation='linear')(HL_D3_pos)
decoder_pos = tf.keras.Model(decoder_inputs_pos, decoder_outputs_pos)

inputs_pos = tf.keras.Input(shape=input_shape)
encoded_pos = encoder(inputs_pos)
decoded_pos = decoder(encoded_pos)
autoencoder_pos = tf.keras.Model(inputs_pos, decoded_pos)

autoencoder_pos.compile(optimizer='adam', loss='mse')


In [ ]:
autoencoder_pos.fit(X_position_scaled, X_position_scaled, batch_size=3, epochs=1)
autoencoder_pos.fit(X_position_scaled, X_position_scaled, batch_size=32, epochs=10)



55467/55467 [==============================] - 160s 3ms/step - loss: 0.2568
Epoch 1/10
5200/5200 [==============================] - 16s 3ms/step - loss: 0.1527
Epoch 2/10
5200/5200 [==============================] - 15s 3ms/step - loss: 0.1375
Epoch 3/10
5200/5200 [==============================] - 15s 3ms/step - loss: 0.1240
Epoch 4/10
5200/5200 [==============================] - 15s 3ms/step - loss: 0.1129
Epoch 5/10
5200/5200 [==============================] - 15s 3ms/step - loss: 0.1035
Epoch 6/10
5200/5200 [==============================] - 15s 3ms/step - loss: 0.0963
Epoch 7/10
5200/5200 [==============================] - 15s 3ms/step - loss: 0.0910
Epoch 8/10
5200/5200 [==============================] - 16s 3ms/step - loss: 0.0870
Epoch 9/10
5200/5200 [==============================] - 16s 3ms/step - loss: 0.0837
Epoch 10/10
5200/5200 [==============================] - 16s 3ms/step - loss: 0.0804


In [ ]:
for layer in encoder.layers:
    layer.trainable = False
for layer in encoder_pos.layers:
    layer.trainable = False

concatenated = tf.keras.layers.Concatenate()([encoder.output, encoder_pos.output])

inputs3 = tf.keras.Input(shape=(10 + 10,))
L1 = tf.keras.layers.Dense(20, activation='elu')(inputs3)
L2 = tf.keras.layers.Dense(20, activation='elu')(L1)
L3 = tf.keras.layers.Dense(20, activation='elu')(L2)
L4 = tf.keras.layers.Dense(20, activation='elu')(L3)
L5 = tf.keras.layers.Dense(20, activation='elu')(L4)
L6 = tf.keras.layers.Dense(20, activation='elu')(L5)
L7 = tf.keras.layers.Dense(20, activation='elu')(L6)
L8 = tf.keras.layers.Dense(20, activation='elu')(L7)
outputs3_4 = tf.keras.layers.Dense(13, activation='softmax')(L8)
nn3 = tf.keras.Model(inputs3, outputs3_4)

outputs1 = encoder(encoder_inputs)
outputs2 = encoder_pos(encoder_inputs_pos)
concatenated = tf.keras.layers.Concatenate()([outputs1, outputs2])
outputs3 = nn3(concatenated)
full_model = tf.keras.Model([encoder_inputs, encoder_inputs_pos], outputs3)


# full_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
full_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])



In [8]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_encoded = le.fit_transform(Y)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# autoencoder_pos.fit(X_position_scaled, X_position_scaled, batch_size=3, epochs=1)
# autoencoder_pos.fit(X_position_scaled, X_position_scaled, batch_size=32, epochs=10)

one_hot_labels = tf.keras.utils.to_categorical(y_encoded)

full_model.fit([X_force_scaled, X_position_scaled], one_hot_labels, batch_size=5, epochs=1)
full_model.fit([X_force_scaled, X_position_scaled], one_hot_labels, batch_size=32, epochs=10)


33280/33280 [==============================] - 107s 3ms/step - loss: 1.0779 - categorical_accuracy: 0.6068
Epoch 1/10
5200/5200 [==============================] - 19s 3ms/step - loss: 0.6271 - categorical_accuracy: 0.7786
Epoch 2/10
5200/5200 [==============================] - 18s 4ms/step - loss: 0.5438 - categorical_accuracy: 0.8068
Epoch 3/10
5200/5200 [==============================] - 19s 4ms/step - loss: 0.4751 - categorical_accuracy: 0.8297
Epoch 4/10
5200/5200 [==============================] - 19s 4ms/step - loss: 0.4343 - categorical_accuracy: 0.8427
Epoch 5/10
5200/5200 [==============================] - 17s 3ms/step - loss: 0.4002 - categorical_accuracy: 0.8546
Epoch 6/10
5200/5200 [==============================] - 18s 3ms/step - loss: 0.3776 - categorical_accuracy: 0.8633
Epoch 7/10
5200/5200 [==============================] - 18s 4ms/step - loss: 0.3588 - categorical_accuracy: 0.8695
Epoch 8/10
5200/5200 [==============================] - 18s 3ms/step - loss: 0.3430 - ca

In [ ]:

full_model.fit([X_force_scaled, X_position_scaled], one_hot_labels, batch_size=32, epochs=30)

Epoch 1/30
5200/5200 [==============================] - 20s 4ms/step - loss: 0.3077 - categorical_accuracy: 0.8877
Epoch 2/30
5200/5200 [==============================] - 18s 3ms/step - loss: 0.3009 - categorical_accuracy: 0.8904
Epoch 3/30
5200/5200 [==============================] - 17s 3ms/step - loss: 0.2912 - categorical_accuracy: 0.8938
Epoch 4/30
5200/5200 [==============================] - 18s 3ms/step - loss: 0.2846 - categorical_accuracy: 0.8963
Epoch 5/30
5200/5200 [==============================] - 20s 4ms/step - loss: 0.2731 - categorical_accuracy: 0.9002
Epoch 6/30
5200/5200 [==============================] - 18s 3ms/step - loss: 0.2729 - categorical_accuracy: 0.9006
Epoch 7/30
5200/5200 [==============================] - 18s 3ms/step - loss: 0.2659 - categorical_accuracy: 0.9028
Epoch 8/30
5200/5200 [==============================] - 18s 3ms/step - loss: 0.2606 - categorical_accuracy: 0.9059
Epoch 9/30
5200/5200 [==============================] - 19s 4ms/step - loss: 0.2

#CNN

In [11]:
import tensorflow as tf

input_shape = (16,1)

encoder_inputs = tf.keras.Input(shape=input_shape)
HL1 = tf.keras.layers.Convolution1D(32,1, input_shape=(16,1), activation='relu')(encoder_inputs)
HL2 = tf.keras.layers.Convolution1D(16,1, activation='relu')(HL1)
HLF = tf.keras.layers.Flatten()(HL2)
HL3 = tf.keras.layers.Dense(24, activation='relu')(HLF)
HL4 = tf.keras.layers.Dense(18, activation='relu')(HL3)
HL5 = tf.keras.layers.Dense(10, activation='relu')(HL4)
encoder = tf.keras.Model(encoder_inputs, HL5)

decoder_inputs = tf.keras.Input(shape=(10,))
HL_D1 = tf.keras.layers.Dense(16, activation='sigmoid')(decoder_inputs)
HL_D2 = tf.keras.layers.Dense(16, activation='sigmoid')(HL_D1)
HL_D3 = tf.keras.layers.Dense(16, activation='sigmoid')(HL_D2)
decoder_outputs = tf.keras.layers.Dense(16, activation='linear')(HL_D3)
decoder = tf.keras.Model(decoder_inputs, decoder_outputs)

inputs = tf.keras.Input(shape=input_shape)
encoded = encoder(inputs)
decoded = decoder(encoded)
autoencoder = tf.keras.Model(inputs, decoded)

autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.fit(X_force_scaled, X_force_scaled, batch_size=3, epochs=1)
autoencoder.fit(X_force_scaled, X_force_scaled, batch_size=32, epochs=10)




55467/55467 [==============================] - 163s 3ms/step - loss: 0.1574
Epoch 1/10
5200/5200 [==============================] - 15s 3ms/step - loss: 0.0725
Epoch 2/10
5200/5200 [==============================] - 16s 3ms/step - loss: 0.0593
Epoch 3/10
5200/5200 [==============================] - 15s 3ms/step - loss: 0.0477
Epoch 4/10
5200/5200 [==============================] - 15s 3ms/step - loss: 0.0380
Epoch 5/10
5200/5200 [==============================] - 15s 3ms/step - loss: 0.0298
Epoch 6/10
5200/5200 [==============================] - 15s 3ms/step - loss: 0.0275
Epoch 7/10
5200/5200 [==============================] - 15s 3ms/step - loss: 0.0204
Epoch 8/10
5200/5200 [==============================] - 15s 3ms/step - loss: 0.0180
Epoch 9/10
5200/5200 [==============================] - 16s 3ms/step - loss: 0.0166
Epoch 10/10
5200/5200 [==============================] - 16s 3ms/step - loss: 0.0157


In [13]:
import tensorflow as tf

input_shape = (16,1)

# encoder_inputs_pos = tf.keras.Input(shape=input_shape)
# HL1_pos = tf.keras.layers.Dense(32, activation='sigmoid')(encoder_inputs_pos)
# HL2_pos = tf.keras.layers.Dense(24, activation='sigmoid')(HL1_pos)
# HL3_pos = tf.keras.layers.Dense(24, activation='relu')(HL2_pos)
# HL4_pos = tf.keras.layers.Dense(18, activation='relu')(HL3_pos)
# HL5_pos = tf.keras.layers.Dense(10, activation='relu')(HL4_pos)
# encoder_pos = tf.keras.Model(encoder_inputs_pos, HL5_pos)

encoder_inputs_pos = tf.keras.Input(shape=input_shape)
HL1_pos = tf.keras.layers.Convolution1D(32,1, input_shape=(16,1), activation='relu')(encoder_inputs_pos)
HL2_pos = tf.keras.layers.Convolution1D(16,1, activation='relu')(HL1_pos)
HLF_pos = tf.keras.layers.Flatten()(HL2_pos)
HL3_pos = tf.keras.layers.Dense(24, activation='relu')(HLF_pos)
HL4_pos = tf.keras.layers.Dense(18, activation='relu')(HL3_pos)
HL5_pos = tf.keras.layers.Dense(10, activation='relu')(HL4_pos)
encoder_pos = tf.keras.Model(encoder_inputs_pos, HL5_pos)

decoder_inputs_pos = tf.keras.Input(shape=(10,))
HL_D1_pos = tf.keras.layers.Dense(16, activation='sigmoid')(decoder_inputs_pos)
HL_D2_pos = tf.keras.layers.Dense(16, activation='sigmoid')(HL_D1_pos)
HL_D3_pos = tf.keras.layers.Dense(16, activation='sigmoid')(HL_D2_pos)
decoder_outputs_pos = tf.keras.layers.Dense(16, activation='linear')(HL_D3_pos)
decoder_pos = tf.keras.Model(decoder_inputs_pos, decoder_outputs_pos)

inputs_pos = tf.keras.Input(shape=input_shape)
encoded_pos = encoder(inputs_pos)
decoded_pos = decoder(encoded_pos)
autoencoder_pos = tf.keras.Model(inputs_pos, decoded_pos)

autoencoder_pos.compile(optimizer='adam', loss='mse')

autoencoder_pos.fit(X_position_scaled, X_position_scaled, batch_size=3, epochs=1)
autoencoder_pos.fit(X_position_scaled, X_position_scaled, batch_size=32, epochs=10)

55467/55467 [==============================] - 147s 3ms/step - loss: 0.2588
Epoch 1/10
5200/5200 [==============================] - 16s 3ms/step - loss: 0.1541
Epoch 2/10
5200/5200 [==============================] - 15s 3ms/step - loss: 0.1408
Epoch 3/10
5200/5200 [==============================] - 15s 3ms/step - loss: 0.1315
Epoch 4/10
5200/5200 [==============================] - 15s 3ms/step - loss: 0.1245
Epoch 5/10
5200/5200 [==============================] - 15s 3ms/step - loss: 0.1158
Epoch 6/10
5200/5200 [==============================] - 15s 3ms/step - loss: 0.1085
Epoch 7/10
5200/5200 [==============================] - 15s 3ms/step - loss: 0.1032
Epoch 8/10
5200/5200 [==============================] - 15s 3ms/step - loss: 0.0986
Epoch 9/10
5200/5200 [==============================] - 17s 3ms/step - loss: 0.0938
Epoch 10/10
5200/5200 [==============================] - 17s 3ms/step - loss: 0.0893


In [14]:
for layer in encoder.layers:
    layer.trainable = False
for layer in encoder_pos.layers:
    layer.trainable = False

concatenated = tf.keras.layers.Concatenate()([encoder.output, encoder_pos.output])

inputs3 = tf.keras.Input(shape=(10 + 10,))
L1 = tf.keras.layers.Dense(20, activation='elu')(inputs3)
L2 = tf.keras.layers.Dense(20, activation='elu')(L1)
L3 = tf.keras.layers.Dense(20, activation='elu')(L2)
L4 = tf.keras.layers.Dense(20, activation='elu')(L3)
L5 = tf.keras.layers.Dense(20, activation='elu')(L4)
L6 = tf.keras.layers.Dense(20, activation='elu')(L5)
L7 = tf.keras.layers.Dense(20, activation='elu')(L6)
L8 = tf.keras.layers.Dense(20, activation='elu')(L7)
outputs3_4 = tf.keras.layers.Dense(13, activation='softmax')(L8)
nn3 = tf.keras.Model(inputs3, outputs3_4)

outputs1 = encoder(encoder_inputs)
outputs2 = encoder_pos(encoder_inputs_pos)
concatenated = tf.keras.layers.Concatenate()([outputs1, outputs2])
outputs3 = nn3(concatenated)
full_model = tf.keras.Model([encoder_inputs, encoder_inputs_pos], outputs3)


# full_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
full_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])



In [15]:
# autoencoder_pos.fit(X_position_scaled, X_position_scaled, batch_size=3, epochs=1)
# autoencoder_pos.fit(X_position_scaled, X_position_scaled, batch_size=32, epochs=10)

one_hot_labels = tf.keras.utils.to_categorical(y_encoded)

full_model.fit([X_force_scaled, X_position_scaled], one_hot_labels, batch_size=5, epochs=1)
full_model.fit([X_force_scaled, X_position_scaled], one_hot_labels, batch_size=32, epochs=10)


33280/33280 [==============================] - 92s 3ms/step - loss: 0.5905 - categorical_accuracy: 0.7980
Epoch 1/10
5200/5200 [==============================] - 17s 3ms/step - loss: 0.2666 - categorical_accuracy: 0.9089
Epoch 2/10
5200/5200 [==============================] - 15s 3ms/step - loss: 0.2231 - categorical_accuracy: 0.9232
Epoch 3/10
5200/5200 [==============================] - 16s 3ms/step - loss: 0.1894 - categorical_accuracy: 0.9351
Epoch 4/10
5200/5200 [==============================] - 16s 3ms/step - loss: 0.1668 - categorical_accuracy: 0.9439
Epoch 5/10
5200/5200 [==============================] - 16s 3ms/step - loss: 0.1513 - categorical_accuracy: 0.9487
Epoch 6/10
5200/5200 [==============================] - 16s 3ms/step - loss: 0.1400 - categorical_accuracy: 0.9529
Epoch 7/10
5200/5200 [==============================] - 16s 3ms/step - loss: 0.1304 - categorical_accuracy: 0.9560
Epoch 8/10
5200/5200 [==============================] - 16s 3ms/step - loss: 0.1238 - cat